In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool
import seaborn as sns

In [ ]:
df_train_0 = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
df_train_0

In [ ]:
corr = df_train_0.corr()

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap='Blues', axis=1)\
    .set_properties(**{'max-width': '200px', 'font-size': '10pt'})\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
df_train = df_train_0
df_train[['Cabin_deck','Cabin_num','Cabin_side']] = df_train.Cabin.str.split('/',expand=True)
df_train[['First_Name','Second_name']] = df_train.Name.str.split(' ',expand=True)
#df_train.Cabin = df_train.Cabin.astype('category').cat.codes
#df_train = df_train.drop(['PassengerId'], axis=1)
df_train

In [ ]:
type(df_train.groupby(['Cabin', 'Second_name'])['PassengerId'].count())

In [ ]:
#vc = df_train['Cabin'].value_counts()
#tg = df_train.groupby(['Cabin', 'Second_name'])['PassengerId'].count()
#df_train['Is_alone'] = df_train['Cabin'].map(lambda x: 0 if x in vc.index and vc.loc[x]>1 else 1)
#df_train['Fam_num'] = df_train.apply(lambda x: tg.loc[(x.Cabin, x.Second_name)] if (x.Cabin, x.Second_name) in tg.index else 1, axis=1)
#df_train['Is_Adult'] = df_train.Age.map(lambda x: 1 if x >= 18 else 0)
df_train = df_train.drop(['PassengerId', 'Name', 'First_Name', 'Second_name', 'Cabin', 'Cabin_num'], axis=1)
df_train

In [ ]:
corr = df_train.corr()

corr.style.background_gradient(cmap='Blues', axis=1)\
    .set_properties(**{'max-width': '200px', 'font-size': '10pt'})\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.dtypes

Converting string and boolean categories to int

In [ ]:
def cat_codes(df, cols):
    for col in cols:
        df[col] = df[col].astype('category').cat.codes
    return df;

In [ ]:
cols_to_cat = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin_deck', 'Cabin_side']
df_train = cat_codes(df_train, cols_to_cat)

In [ ]:
df_train.dtypes

Checking class balance:

In [ ]:
df_train['Transported'].value_counts()

Classes are balanced

In [ ]:
x = df_train.drop(['Transported'], axis=1)
y = df_train.Transported

Imputing missing values:

In [ ]:
imputer = KNNImputer(n_neighbors=3)
x_imp = imputer.fit_transform(x)
x_imp = pd.DataFrame(x_imp, columns = x.columns)

In [ ]:
def str_to_codes(df, cols):
    for col in cols:
        df[col] = df[col].astype('int8').astype('category')
    return df;

x_imp = str_to_codes(x_imp, cols_to_cat)

In [ ]:
x_imp

In [ ]:
x_imp['Age'].plot.kde(bw_method=0.3)

In [ ]:
x_imp['RoomService'].plot.kde(bw_method=0.3)

In [ ]:
x_imp['FoodCourt'].plot.kde(bw_method=0.3)

In [ ]:
x_imp['ShoppingMall'].plot.kde(bw_method=0.3)

In [ ]:
x_imp['Spa'].plot.kde(bw_method=0.3)

In [ ]:
#best result - 0.78933
#clf = RandomForestClassifier(max_depth=100, random_state=0)
#clf.fit(x_imp, y)

In [ ]:
#best result - 0.80406
#gbc = GradientBoostingClassifier()

In [ ]:
#best result - 0.79705
#cbc = CatBoostClassifier(loss_function='CrossEntropy',
#                           verbose=1000, cat_features=cols_to_cat, random_strength=10)

In [ ]:
#parameters = {
#    "n_estimators":[20,50,100,150,200,300],
#    "max_depth":[3,6,7,8,10],
#    "learning_rate":[0.01,0.05,0.1,0.5]
#}

In [ ]:
#cv = GridSearchCV(gbc,parameters,cv=5)
#cv.fit(x_imp,y)

In [ ]:
#cv.best_params_

In [ ]:
#grid_search_result = cbc.grid_search(parameters, x_imp, y.astype('int8'), plot=True)

In [ ]:
grid_search_result['params']

In [ ]:
#cbc = CatBoostClassifier(loss_function='Logloss',
#                         cat_features=cols_to_cat,
#                         iterations=1000,
#                         depth=7,
#                         learning_rate=0.1,
#                         verbose=False)
#cbc.fit(x_imp, y)

In [ ]:
#cbc.get_feature_importance()

In [ ]:
gbc = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, n_estimators=500)
gbc.fit(x_imp, y)

Making prediction

In [ ]:
df_test_0 = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_test_0

In [ ]:
df_test = df_test_0
df_test[['Cabin_deck','Cabin_num','Cabin_side']] = df_test.Cabin.str.split('/',expand=True)
df_test[['First_Name','Second_name']] = df_test.Name.str.split(' ',expand=True)
#df_test.Cabin = df_test.Cabin.astype('category').cat.codes
#vc = df_test['Cabin'].value_counts()
#tg = df_test.groupby(['Cabin', 'Second_name'])['PassengerId'].count()
#df_test['Is_alone'] = df_test['Cabin'].map(lambda x: 0 if x in vc.index and vc.loc[x]>1 else 1)
#df_test['Fam_num'] = df_test.apply(lambda x: tg.loc[(x.Cabin, x.Second_name)] if (x.Cabin, x.Second_name) in tg.index else 1, axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'First_Name', 'Second_name', 'Cabin', 'Cabin_num'], axis=1)
df_test = cat_codes(df_test, cols_to_cat)
imputer = KNNImputer(n_neighbors=2)
X_pr = imputer.fit_transform(df_test)
X_pr = pd.DataFrame(X_pr, columns = df_test.columns)
X_pr = str_to_codes(X_pr, cols_to_cat)
X_pr

In [ ]:
y_pred = gbc.predict(X_pr)

In [ ]:
y_pred

In [ ]:
out_df = df_test_0
out_df['Transported'] = pd.Series(y_pred)
out_df = out_df[['PassengerId','Transported']]
out_df.to_csv('submission_18.csv', index=False)
out_df